In [0]:
%run "/sales_DWH/Includes/Common_function"

In [0]:
## Init access and functions
from pyspark.sql import functions as F
from pyspark.sql.window import Window
get_access_data_lake()

In [0]:
# This class ETL process the silver layer erp_px_cat_g1v2
class silver_erp_px_cat_g1v2():
    def __init__(self):
        # Base path of the Bronze data in ADLS

        self.base_path = "abfss://bronze@salesdwh.dfs.core.windows.net/"

    def read_data(self):
        # Read data from the specified path in Parquet format
        # Using the schema defined above

        df=spark.read.format("parquet").option("inferSchema", "true").option("samplingRatio", 0.01).option('header','true')\
            .load(f"{self.base_path}/erp_px_cat_g1v2/")
        return df
    
    def filter_data(self,df):
        max_month = df.agg(F.max("month").alias("max_month")).collect()[0]["max_month"]
        df_filter = df.filter(F.col("month") == max_month)
        df_scound=df_filter.dropDuplicates().dropna()
        return df_scound
    def write_data(self,df):
        # Define merge condition for incremental load
        merge_condition = "tgt.id=src.id " 
        
        # Call incremental load function to merge data into Silver table
        load=icremental_load(df,catalog_name="salesdwh_catalog",schema_name="silver",table_name="erp_px_cat_g1v2",merge_condition=merge_condition)
        return load
    
    def run(self):
        # Run the full ETL process for silver erp_px_cat_g1v2s data
        print("Starting  Transformation silver_erp_px_cat_g1v2......")
        read_data=self.read_data() 
        data_filter=self.filter_data(read_data)                             # Step 1: Read data
        write_data=self.write_data(data_filter)              # Step 3: Write to Silver layer
        print('Done...!!!') 
        return write_data



In [0]:
## Trigger Transformation
start=silver_erp_px_cat_g1v2()
start.run()

Starting  Transformation silver_erp_px_cat_g1v2......
Done...!!!


' Merge completed: [The affected rows = 37]-----[The updated rows = 37]----[The inserted rows =0] '

In [0]:
%sql
select * from salesdwh_catalog.silver.erp_px_cat_g1v2